In [43]:
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold,KFold,ShuffleSplit
import cntk as c
import matplotlib.pyplot as plt
from keras import backend as c
from keras import layers
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import SGD, Adam
from keras import regularizers
from keras.layers import Input, Dense
from keras import losses
from keras.models import Model
from keras.layers import Input, Dense

In [44]:
x_train= np.loadtxt('C:/Users/Devanshu/Desktop/faltu/data without misslabel/train_test_without_miss_normalize.csv', delimiter=',', dtype='float32')
x_train_label= np.loadtxt('C:/Users/Devanshu/Desktop/faltu/data without misslabel/label_encoded_train.csv', delimiter=',', dtype='float32')
x_test_label= np.loadtxt('C:/Users/Devanshu/Desktop/faltu/data without misslabel/label_encoded_test.csv', delimiter=',', dtype='float32')
print(x_train.shape)
print(x_train_label.shape)
print(x_test_label.shape)

(144766, 40)
(125973, 5)
(18793, 5)


In [45]:
train= x_train[0:125973,:]
test= x_train[125973:,:]
train_label= x_train_label
print(train.shape,test.shape)
#print(train.shape, train_label.shape,test.shape)

(125973, 40) (18793, 40)


In [97]:
# layer 1
input_ly_1= Input(shape=(40,))
enc_ly_1= Dense(30, activation='relu')(input_ly_1)
dec_ly_1=Dense(40, activation='relu')(enc_ly_1)

auto_ly_1= Model(input_ly_1, dec_ly_1)
encoder_ly_1= Model(input_ly_1, enc_ly_1)

auto_ly_1.compile(optimizer='adam', loss='mean_squared_error')
auto_ly_1.fit(train,train,epochs=1,validation_split=0.2)


ly1_predict= encoder.predict(train)
print(ly1_predict.shape)

Train on 100778 samples, validate on 25195 samples
Epoch 1/1
100778/100778 [==============================] - 19s 191us/step - loss: 0.0065 - val_loss: 0.0021
(125973, 30)


In [99]:
#layer 2
input_ly_2= Input(shape=(30,))
enc_ly_2= Dense(15, activation='relu')(input_ly_2)
dec_ly_2= Dense(30, activation='relu')(enc_ly_2)

auto_ly_2= Model(input_ly_2, dec_ly_2)
encoder_ly_2= Model(input_ly_2, enc_ly_2)

auto_ly_2.compile(optimizer='adam', loss='mean_squared_error')
auto_ly_2.fit(ly1_predict,ly1_predict,epochs=1,validation_split=0.2)

ly2_predict= encoder_ly_2.predict(ly1_predict)
print(ly2_predict.shape)

Train on 100778 samples, validate on 25195 samples
Epoch 1/1
100778/100778 [==============================] - 18s 176us/step - loss: 0.0956 - val_loss: 0.0869
(125973, 15)


In [101]:
#layer 3
input_ly_3= Input(shape=(15,))
enc_ly_3= Dense(5, activation='relu')(input_ly_3)
dec_ly_3= Dense(15, activation='relu')(enc_ly_3)

auto_ly_3= Model(input_ly_3, dec_ly_3)
encoder_ly_3= Model(input_ly_3, enc_ly_3)

auto_ly_3.compile(optimizer='adam', loss='mean_squared_error')
auto_ly_3.fit(ly2_predict,ly2_predict,epochs=1,validation_split=0.2)

ly3_predict= encoder_ly_3.predict(ly2_predict)
print(ly3_predict.shape)

Train on 100778 samples, validate on 25195 samples
Epoch 1/1
100778/100778 [==============================] - 17s 167us/step - loss: 0.3416 - val_loss: 0.3052
(125973, 5)


In [132]:
# Deep stacked auto encoder
ly_30= auto_ly_1.layers[1].get_weights()
ly_15= auto_ly_2.layers[1].get_weights()
ly_5= auto_ly_3.layers[1].get_weights()


input_da= Input(shape=(40,))
da_1=Dense(30, activation='relu', input_shape=(40,), name="layer1")(input_da)
da_2= Dense(15, activation='relu',name="layer2")(da_1)
da_3= Dense(5, activation='softmax',)(da_2)
da_4= Dense(15, activation='relu',)(da_3)
da_5= Dense(30, activation='relu',)(da_4)
da_6= Dense(40, activation='relu',)(da_5)

da= Model(input_da, da_6)
da_2= Model(input_da, da_3)

da.layers[1].set_weights(ly_30) # first dense layer
da.layers[2].set_weights(ly_15)
da.layers[3].set_weights(ly_5)

da.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
da.fit(train, train,epochs=10,batch_size=128,shuffle=True,verbose=1, validation_split=0.2)

da_2.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
da_2.fit(train, train_label,epochs=10,batch_size=128,shuffle=True,verbose=1, validation_split=0.2)
da_2.evaluate(test,x_test_label)
#sdg= da_2.predict(train)
#np.savetxt('C:/Users/Devanshu/Desktop/faltu/predict_layer_autostack_train.csv', sdg, delimiter=',')

#deep_autoencoder.layers[1].set_weights(autoencoder1.layers[2].get_weights()) # first dense layer

Train on 100778 samples, validate on 25195 samples
Epoch 1/10
100778/100778 [==============================] - 11s 108us/step - loss: 0.0447 - acc: 0.3005 - val_loss: 0.0262 - val_acc: 0.5394
Epoch 2/10
100778/100778 [==============================] - 10s 98us/step - loss: 0.0229 - acc: 0.3861 - val_loss: 0.0208 - val_acc: 0.3906
Epoch 3/10
100778/100778 [==============================] - 11s 105us/step - loss: 0.0200 - acc: 0.2864 - val_loss: 0.0194 - val_acc: 0.2053
Epoch 4/10
100778/100778 [==============================] - 10s 96us/step - loss: 0.0190 - acc: 0.2812 - val_loss: 0.0188 - val_acc: 0.3286
Epoch 5/10
100778/100778 [==============================] - 10s 102us/step - loss: 0.0186 - acc: 0.2760 - val_loss: 0.0185 - val_acc: 0.2897
Epoch 6/10
100778/100778 [==============================] - 10s 102us/step - loss: 0.0184 - acc: 0.2785 - val_loss: 0.0184 - val_acc: 0.1611
Epoch 7/10
100778/100778 [==============================] - 11s 108us/step - loss: 0.0183 - acc: 0.2815 -

[0.0836193269437912, 0.7656042143446397]

Just for reference. Code below is not for use.

In [ ]:
model2= Sequential()
model2.add(Dense(15, activation='relu', input_shape=(30,), name="layer2"))
ly= model1.layers[0].get_weights()
#model2.add(Dense(15, activation='relu', input_shape=(30,), name="layer2"))
model2.add(Dense(30, activation='relu'))

model2.compile(optimizer='adam', loss='mean_squared_error')
model2.fit(train, train,epochs=5,batch_size=128,shuffle=True,verbose=1, validation_split=0.2)


In [55]:
model1= Sequential()
input_img = Input(shape=(40,))
model1.add(Dense(30, activation='relu', input_shape=(40,), name="layer1"))
ly= model1.layers[0].get_weights()
model1.add(Dense(15, activation='relu',name="layer2",trainable=False))
model1.add(Dense(5, activation='relu',trainable=False))

model1.compile(optimizer='adam', loss='mean_squared_error')
model1.fit(train, train,epochs=5,batch_size=128,shuffle=True,verbose=1, validation_split=0.2)

Train on 100778 samples, validate on 25195 samples
Epoch 1/5
100778/100778 [==============================] - 8s 82us/step - loss: 0.0495 - val_loss: 0.0314
Epoch 2/5
100778/100778 [==============================] - 7s 69us/step - loss: 0.0289 - val_loss: 0.0272
Epoch 3/5
100778/100778 [==============================] - 7s 70us/step - loss: 0.0264 - val_loss: 0.0257
Epoch 4/5
100778/100778 [==============================] - 7s 69us/step - loss: 0.0253 - val_loss: 0.0249
Epoch 5/5
100778/100778 [==============================] - 7s 72us/step - loss: 0.0247 - val_loss: 0.0244


In [54]:
model2= Sequential()
input_img = Input(shape=(40,))
model2.add(Dense(30, activation='relu', input_shape=(40,), name="layer1"))
ly= model2.layers[0].get_weights()
model2.add(Dense(15, activation='relu',name="layer2",trainable=False))
model2.add(Dense(5, activation='relu',trainable=False))

model2.compile(optimizer='adam', loss='mean_squared_error')
model.fit(train, train,epochs=5,batch_size=128,shuffle=True,verbose=1, validation_split=0.2)

Train on 100778 samples, validate on 25195 samples
Epoch 1/10
100778/100778 [==============================] - 7s 66us/step - loss: 0.0275 - acc: 0.9118 - val_loss: 0.0109 - val_acc: 0.9692
Epoch 2/10
100778/100778 [==============================] - 7s 66us/step - loss: 0.0097 - acc: 0.9723 - val_loss: 0.0088 - val_acc: 0.9758
Epoch 3/10
100778/100778 [==============================] - 7s 67us/step - loss: 0.0082 - acc: 0.9756 - val_loss: 0.0074 - val_acc: 0.9775
Epoch 4/10
100778/100778 [==============================] - 7s 69us/step - loss: 0.0072 - acc: 0.9798 - val_loss: 0.0066 - val_acc: 0.9819
Epoch 5/10
100778/100778 [==============================] - 7s 66us/step - loss: 0.0066 - acc: 0.9820 - val_loss: 0.0064 - val_acc: 0.9828
Epoch 6/10
100778/100778 [==============================] - 7s 65us/step - loss: 0.0063 - acc: 0.9827 - val_loss: 0.0060 - val_acc: 0.9835
Epoch 7/10
100778/100778 [==============================] - 7s 66us/step - loss: 0.0061 - acc: 0.9833 - val_loss: 0

In [ ]:
model3= Sequential()
input_img = Input(shape=(40,))
model3.add(Dense(30, activation='relu', input_shape=(40,), name="layer1"))
ly= model1.layers[0].get_weights()
model1.add(Dense(15, activation='relu',name="layer2",trainable=False))
model1.add(Dense(5, activation='relu',trainable=False))

model1.compile(optimizer='adam', loss='mean_squared_error')
model1.fit(train, train,epochs=5,batch_size=128,shuffle=True,verbose=1, validation_split=0.2)